In [78]:
import pandas as pd
import numpy as np
import re

In [79]:
jobs = pd.read_excel('DataSource_NZSeek/NZ_Admin_JOBS.xlsx', header=None, skiprows=1)

In [80]:
jobs.tail()

,0,1,2,3,4,5
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,location: WellingtonWellingtonarea: Wellington...,"27d ago,at",Competitive hourly rate
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"20d ago,at",Competitive hourly rates $$
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,location: WaikatoWaikatoarea: HamiltonHamilton...,"28d ago,at",Competitive hourly rates $$
2707,Executive Assistant,https://www.seek.co.nz/job/50496571?type=stand...,Alpha Recruitment - NZ,location: WellingtonWellington,"26d ago,at",classification: Administration & Office Suppor...


In [81]:
salary = pd.concat([jobs[3], jobs[5]], axis=1)

In [82]:
salary.columns = ['s1', 's2']

In [83]:
salary

,s1,s2
0,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...
1,location: Bay of PlentyBay of Plentyarea: Taur...,classification: Administration & Office Suppor...
2,location: AucklandAuckland,classification: Administration & Office Suppor...
3,location: SouthlandSouthlandarea: Invercargill...,classification: Administration & Office Suppor...
4,location: CanterburyCanterburyarea: Christchur...,classification: Administration & Office Suppor...
...,...,...
2703,location: AucklandAucklandarea: Auckland Centr...,classification: Administration & Office Suppor...
2704,location: WellingtonWellingtonarea: Wellington...,Competitive hourly rate
2705,location: WaikatoWaikatoarea: HamiltonHamilton...,Competitive hourly rates $$
2706,location: WaikatoWaikatoarea: HamiltonHamilton...,Competitive hourly rates $$


In [84]:
salary['s2'] = salary['s2'].apply(lambda x: np.nan if x.startswith('classification') else x.strip())
salary['s1'] = salary['s1'].where(salary['s2'].notnull(), np.nan)

In [127]:
s = pd.DataFrame(salary['s2'])

In [136]:
s[s['s2'].notnull() & s['s2'].str.contains(' per ')]

,s2
12,$20 per hour
17,$20 - $24.99 per hour
23,$19 to $22 per hour
25,$20 - $24.99 per hour
36,$20 - $24.99 per hour
48,$25 per hour
58,$60 000 - $70 000 per annum
59,$20.00 per hour
66,$20 - $24.99 per hour
77,$20.00 per hour.


In [182]:
import re


def salary_extraction_with_per(x):
    if x != np.nan and ' per ' in x:
        x = x.strip()
        numbers = re.findall(r'\s*\d+.?\d*', x)
        # numbers = re.findall(r'\s*\d+(?:[,\s]\d+)*(?:.\d+)?', x)
        # numbers = [re.sub(',', '', num) for num in numbers]
        numbers = [num.replace(' ', '') for num in numbers]
        print(numbers)
        if len(numbers) == 1:
            low, high = round(float(numbers[0]), 2), round(float(numbers[0]), 2)
        elif len(numbers) == 2:
            low = round(float(numbers[0]), 2)
            high = round(float(numbers[1]), 2)
        else:
            low, high = np.nan, np.nan
        if 'annum' in x:
            # 年薪转化为时薪
            low = low / 52 / 40
            high = high / 52 / 40

            low = round(low, 2)
            high = round(high, 2)
        return low, high

    else:
        return np.nan, np.nan

In [183]:
salary_extraction_with_per('$20 - $24.99 per hour')

['20', '24.99']


(20.0, 24.99)

In [179]:
salary_extraction_with_per('$60 000 - $70 000 per annum')

['60000', '70000']


(28.85, 33.65)

In [180]:
salary_extraction_with_per('$20.00 per hour.')

['20.00']


(20.0, 20.0)

In [165]:
s['s2']

0                               NaN
1                               NaN
2                               NaN
3                               NaN
4                               NaN
                   ...             
2703                            NaN
2704        Competitive hourly rate
2705    Competitive hourly rates $$
2706    Competitive hourly rates $$
2707                            NaN
Name: s2, Length: 2708, dtype: object